# Importing and inspecting the data

We will start by importing the dataset

In [3]:
import pandas as pd
import numpy as np
# Set environment
pd.set_option('max_colwidth', 100)
pd.set_option('precision', 3)
# Read data
df_o=pd.read_csv('./data/train.csv',dtype={'question1':str,'question2':str})

df_o['question1']=df_o['question1'].astype(str)
df_o['question2']=df_o['question2'].astype(str)

# Version of pandas (0.19.2, numpy 1.12.1)
print (pd.__version__)
print (np.__version__)
#Inspect data
df_o.head()
df_o['is_duplicate'].value_counts(normalize=True)*100

0.19.2
1.12.1


0    63.08
1    36.92
Name: is_duplicate, dtype: float64

In [5]:
df_o.dtypes

id               int64
qid1             int64
qid2             int64
question1       object
question2       object
is_duplicate     int64
dtype: object

In [4]:
df_o.drop_duplicates(inplace=True)
df_o.dropna(inplace=True)

In [5]:
df_o.shape

(404290, 6)

## Split data

In [6]:
# Split dataset in training and validation dataset, 70/30
# Test dataset provided will not be used until the end, so it can be used for final validation
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.3
X_train, X_val, y_train, y_val = train_test_split(df_o.ix[:,0:5], df_o.ix[:,'is_duplicate'], test_size=TEST_SIZE, random_state=42)

In [7]:
X_train.shape

(283003, 5)

# Generate features

Some ideas to work on:
1. Question lenght in words
2. Shared words
3. Use tf-idf to identify specific words on each pairs
4. Is question clause the same?
5. Syntax similarity: POS tags?
6. Shared synonims (all words or just the key words- verb, object)
7. Similarity of key words: wordnet/synnet similarity score

In [8]:
# Clean up functions
# Remove stopwords
from nltk.corpus import stopwords
from nltk import tokenize

from nltk.corpus import stopwords
import string

def clean_text(s):
    # call with a string and return a list of words excluding stop words and punctuation and lower case
    
    tok=tokenize.word_tokenize(s)
    s1=[w.lower() for w in tok if w not in stopwords.words('english')]
    
    punct=set(string.punctuation)
    s2=[w for w in s1 if w not in punct]
    return s2
    


#### Feature: different length in words

In [9]:
def dif_len(l1,l2):
    return(abs(len(l1.split())-len(l2.split())))


#### Feature: Number of common words

#### Feature: Use tf-idf with cosine similarity
The approach is to identify "vectorize" the strings using tf idf and then find the similarity between the two.
This feature will produce a number between the 2 sentences that will represent how close they are between each other


In [10]:


# Function to determine the tf idf of a list of strings

from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf(l1, l2):
    # 
    result=[]
   
    for X in zip(l1,l2):
        tfidf_matrix = []
        tfidf_vect=TfidfVectorizer()
        tfidf_sparse=tfidf_vect.fit_transform(X)
        tfidf_matrix=tfidf_sparse.todense()
    
        cosine_distance=cos_sim(tfidf_matrix)
        result.append(cosine_distance)
    return result

# function to identify the cosine similarity between 2 vectors
import sklearn.metrics.pairwise as metrics
def cos_sim(m):
    cos=metrics.cosine_similarity(m[0],m[1])[0,0]
    return cos

In [18]:
q1_list=X_train['question1'].tolist()
q2_list=X_train['question2'].tolist()
id=X_train['id'].tolist()

In [12]:
tfidf_cos=tfidf(q1_list[:],q2_list[:])

In [19]:
tfidf_series=pd.Series(tfidf_cos,index=id)

In [33]:
tfidf_series.head()

20128     0.236
296237    0.433
107095    0.252
27940     0.796
251434    0.127
dtype: float64

#### Feature: common question clause

In [79]:

clause_q1=X_train['question1'].apply(lambda x: x.split()[0])
clause_q2=X_train['question2'].apply(lambda x: x.split()[0])

AttributeError: 'float' object has no attribute 'split'

In [ ]:
clause_q1==clause_q2

#### Feature: Shared synonyms

#### Feature: Similarity of key words: noun and verbs

## Add feature columns to train dataset

### tf idf

In [69]:
X_train.ix[:,'tfdid_cos']=pd.Series(tfidf_cos).values

ValueError: Must have equal len keys and value when setting with an iterable

# Applying models

### Baseline algorithm

In [ ]:
# We will take LogisticRegression as a simple algorithm to establish a baseline


### Other algorithms

In [ ]:
# Flow: 
#   train model with a set of hyperparameters
#   Obtain score and iterate


# Run model on test dataset

In [17]:
pd.__version__

'0.19.2'